In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import hydra
from hydra import compose, initialize

import torchvision
import torchvision.transforms.functional as TF

In [ ]:
with initialize(config_path="../configs/"):
    cfg = compose(config_name="train_vqbet", overrides=["include_task=Bag_Pick_Up"])

In [ ]:
train_dataset = hydra.utils.instantiate(cfg.dataset.train)

In [ ]:
imgs, done, actions = train_dataset[0]